In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms, models
import cv2

In [2]:
inTransform = transforms.Compose([transforms.ToPILImage(),
                                  transforms.Resize(255),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

In [3]:
import os

In [5]:
model = models.densenet121(pretrained=True)

In [6]:
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.5),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.0005)

In [7]:
state_dic = torch.load('./Assets/checkpoint.pth',map_location='cpu')
#print(state_dic.keys())

In [8]:
model.load_state_dict(state_dic)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

# Till Now we've loaded our trained Model ! 

# -------------------------------------------------------------------------------------------

## Now we'll be working on Object Detected video 

In [9]:
import cv2
import pickle

In [10]:
with open("./assets/detectedObjects.txt", "rb") as fp:   # Unpickling
    oList = pickle.load(fp)

In [11]:
len(oList)

167

In [12]:
i = oList[161][0]['box_points']
i

(0, 157, 222, 293)

In [13]:
cap = cv2.VideoCapture('./assets/objectDetectedVideo.avi')
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out = cv2.VideoWriter('./assets/outputTrash.mov',fourcc, 20.0, (640,360))

In [14]:
i=0
font = cv2.FONT_HERSHEY_SIMPLEX

if (cap.isOpened()== False): 
    print("Error opening video stream or file")
while(cap.isOpened()):
   # Capture frame-by-frame
    ret, frame = cap.read()
    if ret == True:
        width = cap.get(3)
        height = cap.get(4)
        #print(width,height)

    # Display the resulting frame
        
        if len(oList[i])!=0:
            for j in range(len(oList[i])):
                c = oList[i][j]['box_points']
                image = frame[c[1]:c[3],c[0]:c[2]]
                inf = inTransform(image)
                inf = inf.unsqueeze(0)
                logps = model.forward(inf)
                ps = torch.exp(logps)
                top_p, top_class = ps.topk(1, dim=1)
                
                if ps[0][1]>0.7:
                    cv2.putText(frame,'Accident !',(c[0]-10,c[1]-10), font, 1.5,(0,0,255),2,cv2.LINE_AA)
            

        out.write(frame)
        cv2.imshow('Frame',frame)
            
        

    # Press Q on keyboard to  exit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    # Break the loop
    else: 
        break
    i += 1
 
#When everything done, release the video capture object
cap.release()
out.release()
# Closes all the frames
cv2.destroyAllWindows()

# Cheers!